# YOLO v11s를 이용한 개 객체 탐지 후 224x224로 리사이즈하고 모델 테스트
욜로는 COCO 데이터셋의 80개의 클래스를 가지고 사전학습된 모델입니다.
80개의 클래스 중에 16번 인덱스(숫자로 세면 17번쨰)이 개 클래스인데 출처에 따라 인덱스 17이라고 조금 다르게 나옵니다.
일단 작동해서 16으로 두었습니다.

In [63]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image
import cv2
import numpy as np
from ultralytics import YOLO

# 모델, 욜로, 이미지 경로 수정만 해주시면 됩니다.
# ------------------------
# 1. YOLOv11 모델 로드 (개 객체 탐지용)
# ------------------------
# YOLOv11s 모델은 COCO 데이터셋으로 사전 학습되어 있으므로, 'dog' 클래스(일반적으로 클래스 인덱스 16)를 인식합니다.
yolo_model_path = "/Users/vairocana/Downloads/yolo11m.pt"  # YOLOv11s 모델 파일 경로 (해당 파일이 존재해야 함)
yolo_model = YOLO(yolo_model_path)
print("✅ YOLOv11 모델이 성공적으로 로드되었습니다!")

# ------------------------
# 2. ResNet 모델 로드 (분류용)
# ------------------------
model_path = "/Users/vairocana/Desktop/AI/resnet_models/resnet18_model_82.pth"  # 저장된 ResNet50 모델 파일 경로
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print("Using device:", device)

# ResNet50 모델을 사전 학습된 가중치 없이 생성한 후, 마지막 fc 레이어를 데이터셋 클래스 수에 맞게 수정
model = models.resnet18(weights=None)
num_features = model.fc.in_features
num_classes = 10  # 클래스 개수 (데이터셋에 맞게 수정)
model.fc = nn.Linear(num_features, num_classes)
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()  # 평가 모드로 전환 (Dropout, BatchNorm 등 비활성화)
print("✅ ResNet모델이 성공적으로 로드되었습니다!")

# ------------------------
# 3. 이미지 전처리 함수 정의 (224×224, 정규화)
# ------------------------
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 크롭된 이미지를 224×224로 조정
    transforms.ToTensor(),  # 텐서 변환
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet 정규화
])

# ------------------------
# 4. YOLO를 이용하여 "dog" 객체를 탐지하고 크롭하는 함수
# ------------------------
def detect_and_crop(image_path):
    """
    주어진 이미지 경로에서 YOLOv8을 이용해 'dog' 객체(클래스 인덱스 16)를 탐지하고,
    해당 바운딩 박스 영역을 크롭한 후 224×224로 전처리한 이미지를 반환합니다.
    """
    # 이미지 로드 및 BGR -> RGB 변환 (YOLO는 RGB 사용)
    image = cv2.imread(image_path)
    if image is None:
        print(f"❌ 이미지 로드 실패: {image_path}")
        return None
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # YOLO 추론 수행
    results = yolo_model(image_rgb)

    # 디버그: 탐지된 모든 객체 정보 출력
    print("🔍 검출된 객체들:")
    for result in results:
        for box in result.boxes.data:
            # YOLOv11 결과 tensor를 리스트로 변환하여 [x1, y1, x2, y2, confidence, class] 순서로 얻음
            box_vals = box.tolist()
            x1, y1, x2, y2, conf, cls = box_vals
            print(f"  클래스: {int(cls)}, 신뢰도: {conf:.2f}")

    # 'dog' 객체 추출 (COCO 기준 dog 클래스 인덱스는 일반적으로 16)
    found_dog = False
    for result in results:
        for box in result.boxes.data:
            box_vals = box.tolist()
            x1, y1, x2, y2, conf, cls = box_vals
            # 여기서 dog 클래스 조건: 클래스 인덱스 16, 신뢰도 0.4 이상 (필요에 따라 조정)
            if int(cls) == 16 and conf > 0.4:
                found_dog = True
                # 좌표 정수형 변환
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                # 개 객체 영역 크롭 (RGB 이미지 기준)
                cropped_img = image_rgb[y1:y2, x1:x2]
                # PIL 이미지로 변환
                cropped_img_pil = Image.fromarray(cropped_img)
                # 전처리: 224×224 리사이즈 및 정규화, 배치 차원 추가
                processed_img = transform(cropped_img_pil)
                processed_img = processed_img.unsqueeze(0)
                return processed_img.to(device)
    if not found_dog:
        print("⚠️ 이미지에서 개 객체를 감지하지 못했습니다!")
    return None

# ------------------------
# 5. ResNet을 이용하여 포즈(클래스) 분류하는 함수
# ------------------------
def classify_pose(image_path):
    """
    YOLO를 사용해 주어진 이미지에서 개 객체를 탐지하여 크롭한 후,
    ResNet 모델로 분류하여 포즈(클래스)를 예측하는 함수.
    """
    cropped_image = detect_and_crop(image_path)
    if cropped_image is None:
        print("❌ 예측을 수행할 개 이미지가 없습니다.")
        return

    # ResNet50을 이용한 예측 (gradient 계산 비활성화)
    with torch.no_grad():
        outputs = model(cropped_image)
        _, predicted_class = outputs.max(1)

    # 클래스 라벨 (데이터셋에 맞게 수정)
    class_labels = [
        "BODYLOWER", "BODYSCRATCH", "BODYSHAKE", "FEETUP", 
        "FOOTUP", "LYING", "MOUNTING", "SIT", "TURN", "WALKRUN"
    ]
    predicted_label = class_labels[predicted_class.item()]
    print(f"✅ 예측된 포즈: {predicted_label}")

# ------------------------
# 6. 실행: 단일 이미지에 대해 YOLO로 개 객체 탐지 후, ResNet으로 분류
# ------------------------
image_path = "/Users/vairocana/Downloads/sample7.jpeg"  # 테스트할 이미지 경로 (수정 필요)
classify_pose(image_path)


✅ YOLOv11 모델이 성공적으로 로드되었습니다!
Using device: mps
✅ ResNet모델이 성공적으로 로드되었습니다!

0: 416x640 1 dog, 60.5ms
Speed: 0.9ms preprocess, 60.5ms inference, 0.3ms postprocess per image at shape (1, 3, 416, 640)
🔍 검출된 객체들:
  클래스: 16, 신뢰도: 0.95
✅ 예측된 포즈: SIT


# 욜로로 크롭된 이미지 확인을 위한 코드

In [44]:
import torch
import cv2
import os
from ultralytics import YOLO
from PIL import Image
import numpy as np

#욜로와 이미지 경로만 수정해주시면 됩니다.
# ------------------------
# 1. YOLOv11 모델 로드 
# ------------------------
yolo_model_path = "/Users/vairocana/Downloads/yolo11s.pt"  # YOLOv11s 모델 파일 경로 (해당 파일이 존재해야 함)
yolo_model = YOLO(yolo_model_path)
print("✅ YOLOv11 모델이 성공적으로 로드되었습니다!")

# ------------------------
# 2. 이미지 경로 설정 (필요에 따라 수정)
# ------------------------
image_path = "/Users/vairocana/Downloads/coco_sample2.jpg"
output_path = "/Users/vairocana/Downloads//cropped_dog.jpg"

# ------------------------
# 3. 이미지 로드 및 YOLO 추론 수행
# ------------------------
image = cv2.imread(image_path)
if image is None:
    print(f"❌ 이미지 로드 실패: {image_path}")
    exit()

# OpenCV는 BGR, YOLO는 RGB이므로 변환
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

results = yolo_model(image_rgb)  # YOLO 추론 수행

# ------------------------
# 4. 디버깅: 검출된 모든 객체 정보 출력
# ------------------------
print("🔍 검출된 객체들:")
for result in results:
    # result.boxes.data는 각 박스에 대해 [x1, y1, x2, y2, confidence, class]를 포함하는 tensor입니다.
    for box in result.boxes.data:
        # 텐서를 리스트로 변환하여 인덱스로 접근
        box = box.tolist()
        x1, y1, x2, y2, conf, cls = box
        print(f"  클래스: {int(cls)}, 신뢰도: {conf:.2f}")

# ------------------------
# 5. "dog" 클래스 (COCO 기준 dog의 클래스 인덱스는 16이라고 나온곳도 있고 17도 있음..)만 추출하여 크롭
# ------------------------
found_dog = False
for result in results:
    for box in result.boxes.data:
        box = box.tolist()
        x1, y1, x2, y2, conf, cls = box
        if int(cls) == 16 and conf > 0.4:  # dog 클래스 인덱스가 16 아니면 17인데 찾아도 둘다 나와서 일단 16으로 설정. 조건 (신뢰도 0.4 이상)
            found_dog = True
            # 좌표 정수형 변환
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            # 개 객체 영역 크롭 (RGB 이미지 기준)
            cropped_dog = image_rgb[y1:y2, x1:x2]
            # PIL 이미지로 변환 후 저장 (JPEG 품질 95)
            cropped_img_pil = Image.fromarray(cropped_dog)
            cropped_img_pil.save(output_path, quality=95)
            print(f"✅ 개 객체를 검출하여 크롭했습니다! 저장 경로: {output_path}")
            break  # 하나의 개만 처리
    if found_dog:
        break

if not found_dog:
    print("❌ 이미지에서 개 객체를 감지하지 못했습니다!")


✅ YOLOv11 모델이 성공적으로 로드되었습니다!

0: 640x512 2 dogs, 1 skis, 46.3ms
Speed: 1.3ms preprocess, 46.3ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 512)
🔍 검출된 객체들:
  클래스: 16, 신뢰도: 0.81
  클래스: 16, 신뢰도: 0.36
  클래스: 30, 신뢰도: 0.33
✅ 개 객체를 검출하여 크롭했습니다! 저장 경로: /Users/vairocana/Downloads//cropped_dog.jpg
